# Part01 Data Splitting

In [77]:
# import modules
from typing import List, Dict, NamedTuple, Any

import pandas as pd
from IPython.display import display
import joblib
import yaml
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from src.classes.TrData import TrData
from src.commons.Utils import cleanse_transform_data

In [78]:
raw_data: pd.DataFrame = pd.read_csv(
    "../../dataset/00_raw/teleco_customer_churn.csv"
)

raw_data.head(n=5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


We will not do `Data Exploration` in this notebook. 

Data Exploration has been done in `part00_data_exploration.ipynb` notebook

In this notebook, we will focus on:
* splitting the data into training, validation, and testing set
* calculate mean for `TotalCharge` column from training dataset
* Impute the `null` in `TotalCharge` column with mean
* Save the calculated mean, so that it can be used again in the inference step

In [79]:
churn_label: pd.DataFrame 
df_ohe: pd.DataFrame

_ ,churn_label, df_ohe = cleanse_transform_data(rw_data=raw_data)

print(churn_label.dtypes)
print("--"*6)
print(df_ohe.dtypes)

Churn    int64
dtype: object
------------
tenure                                       int64
MonthlyCharges                             float64
TotalCharges                               float64
gender_Female                                int64
gender_Male                                  int64
SeniorCitizen_0                              int64
SeniorCitizen_1                              int64
Partner_No                                   int64
Partner_Yes                                  int64
Dependents_No                                int64
Dependents_Yes                               int64
PhoneService_No                              int64
PhoneService_Yes                             int64
MultipleLines_No                             int64
MultipleLines_No phone service               int64
MultipleLines_Yes                            int64
InternetService_DSL                          int64
InternetService_Fiber optic                  int64
InternetService_No                      

Let's split the `df_ohe` and `churn_label` into train, validation, and test dataset

In [80]:
ohe_train: pd.DataFrame
ohe_val: pd.DataFrame
ohe_test: pd.DataFrame
churn_train: pd.DataFrame
churn_val: pd.DataFrame
churn_test: pd.DataFrame

ohe_train, ohe_test, churn_train, churn_test = train_test_split(
    df_ohe, 
    churn_label, 
    test_size=0.2, 
    shuffle=True, 
    random_state=200
)

ohe_train, ohe_val, churn_train, churn_val = train_test_split(
    ohe_train,
    churn_train, 
    test_size=0.25,
    shuffle=True, 
    random_state=200
)

print("--"*6)
print(f"ohe_train {ohe_train.shape} | churn_train {churn_train.shape}")
print("--"*6)
print(f"ohe_val {ohe_val.shape} | churn_val {churn_val.shape}")
print("--"*6)
print(f"ohe_val {ohe_test.shape} | churn_val {churn_test.shape}")

dataset_to_save: Dict[str, pd.DataFrame] = {
    "ohe_train": ohe_train, 
    "churn_train": churn_train,
    "ohe_val": ohe_val, 
    "churn_val": churn_val,
    "ohe_test": ohe_test,
    "churn_test": churn_test
}

# save split dataset
for each_key, each_df in dataset_to_save.items():
    each_df.to_csv(
        path_or_buf=f"../../dataset/01_interim/{each_key}.csv", 
        index=False
    )

------------
ohe_train (5346, 46) | churn_train (5346, 1)
------------
ohe_val (1783, 46) | churn_val (1783, 1)
------------
ohe_val (1783, 46) | churn_val (1783, 1)


Cool, so now we already have `train` dataset, `validation` dataset, and `train` dataset

Now, remember from previous notebook `part00_data_exploration`, we obtained 11 `null` in `TotalCharge` column once it was converted to "int64"

Now that we have split the dataset into `train` , `validation` and `test` dataset; we can calculate the mean from the `train` dataset, and then impute all the `null` in all dataset with the calculated mean from the training

We will save this calculated mean into `config.yaml` because we have to impute the same value to future `null` in future inferences

In [81]:
mean_total_charges: float = round(ohe_train["TotalCharges"].mean(), 2) 

# Save calculated mean_total_charges to YAML file for future uses
with open("../../config.yaml", "w") as file_stream:
    yaml.safe_dump(
        data={"MEAN_TOTAL_CHARGES": int(mean_total_charges)},
        stream=file_stream
    )

# impute null values with calculated mean_total_charges
ohe_train.fillna(value=mean_total_charges, inplace=True)

Let's find the range of each column in `ohe_train`

In [82]:
df_range: pd.DataFrame =  ohe_train.agg(func=["min", "max"])

display(df_range)

,tenure,MonthlyCharges,TotalCharges,gender_Female,gender_Male,SeniorCitizen_0,SeniorCitizen_1,Partner_No,Partner_Yes,Dependents_No,...,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
min,0,18.25,18.8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
max,72,118.65,8594.4,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


For those columns whose `max > 1` : we need to scale them so that they are in the same range as other columns

In [83]:
# Find list of columns whose  `Max` is greater than `1`
columns_to_scale: List[str] = df_range.columns[df_range.loc["max"] > 1].to_list() 

In [84]:
mm_scaler: MinMaxScaler = MinMaxScaler(
    feature_range=(0, 1)
)

# train scaler using train dataset
for each_cols in columns_to_scale:
    ds: pd.Series = ohe_train[[each_cols]].copy()
    ds_scaled = mm_scaler.fit_transform(ds)
    joblib.dump(
        value=mm_scaler,
        filename=f"../../models/scaler/mm_scaler_{each_cols}.pkl"
    ) # -->> save each scaler to pickle
    ohe_train[[each_cols]] = ds_scaled
    

display(ohe_train)

,tenure,MonthlyCharges,TotalCharges,gender_Female,gender_Male,SeniorCitizen_0,SeniorCitizen_1,Partner_No,Partner_Yes,Dependents_No,...,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
2228,0.875000,0.401394,0.406351,1,0,1,0,1,0,1,...,1,0,1,0,1,0,0,1,0,0
4264,1.000000,0.659363,0.701327,0,1,1,0,0,1,1,...,1,0,0,1,1,0,0,0,0,1
5549,0.972222,0.504980,0.564380,0,1,1,0,0,1,0,...,0,0,0,1,1,0,0,1,0,0
6960,0.069444,0.276394,0.023660,1,0,1,0,1,0,0,...,0,1,0,0,1,0,0,0,0,1
7286,0.041667,0.517928,0.024593,1,0,1,0,1,0,1,...,0,1,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7832,0.055556,0.537351,0.033438,0,1,1,0,1,0,1,...,0,1,0,0,0,1,1,0,0,0
2163,0.402778,0.368526,0.186739,1,0,1,0,1,0,1,...,0,0,1,0,0,1,0,0,1,0
5479,0.027778,0.257968,0.008612,0,1,0,1,1,0,1,...,0,1,0,0,0,1,0,0,1,0
1070,0.861111,0.815737,0.745703,1,0,1,0,1,0,0,...,1,0,1,0,0,1,0,0,1,0


Alright, once we scale all numerical form to between `0` to `1`, we can convert the dataframe to numpy array for further processing later

In [85]:
# converting pandas dataframe to numpy

ohe_np: np.ndarray = ohe_train.to_numpy()
churn_np: np.ndarray = churn_train.to_numpy()

print(f"ohe_np shape {ohe_np.shape}")
print(f"churn_np shape {churn_np.shape}")

ohe_np shape (5346, 46)
churn_np shape (5346, 1)


Hence, to recap, for the data_splitting, we are doing the following: 
1. impute `null` in `TotalCharges` with mean value
2. Scale `tenure` , `MonthlyCharge`, and `TotalCharges` using MinMaxScaler
3. Convert pandas dataframe to numpy

As usual, let's create a function to encapsulate all these steps, so that it can be reused

In [86]:
class NpData(NamedTuple):
    ohe_np: np.ndarray
    churn_np: np.ndarray

def impute_scale_and_convert_to_numpy(
        ohe_df: pd.DataFrame,
        columns_with_nulls: List[str],
        impute_val: List, 
        scaler_folder: str,
        churn_df: pd.DataFrame
) -> NpData:
    """
    
    :param ohe_df: dataframe to impute and scale 
    :param columns_with_nulls:  the column which has null values
    :param impute_val:  list of values to replace the nulls
    :param scaler_folder:  folder where we put the scaler
    :param churn_df: dataframe of the churn target
    :return: A tuple of scaled feature numpy array and churn target numpy array
    """
    
    # Impute null values
    for each_cols, im_val in zip(columns_with_nulls, impute_val):
        ohe_df[[each_cols]] = ohe_df[[each_cols]].fillna(value=im_val, inplace=False)
    
    # scale columns to between 0 and 1
    df_range: pd.DataFrame = ohe_df.agg(func=["min", "max"])
    columns_to_scale: List[str] = df_range.columns[df_range.loc["max"] > 1].to_list()
    
    for each_cols in columns_to_scale:
        ds: pd.Series = ohe_df[[each_cols]].copy()
        
        file_path: str = f"{scaler_folder}/mm_scaler_{each_cols}.pkl" 
        mm_scaler: MinMaxScaler = joblib.load(filename=file_path)
        ds_scaled = mm_scaler.transform(ds)
        ohe_df[[each_cols]] = ds_scaled
    
    ohe_np: np.ndarray = ohe_df.to_numpy()
    churn_np: np.ndarray = churn_df.to_numpy()
    
    return NpData(
        ohe_np=ohe_np,
        churn_np=churn_np
    )

Let's try our newly minted function 

In [87]:
_ohe_np: np.ndarray
_churn_np: np.ndarray

_ohe_np, _churn_np = impute_scale_and_convert_to_numpy(
    ohe_df=ohe_train,
    columns_with_nulls=["TotalCharges"],
    impute_val=[mean_total_charges],
    scaler_folder="../../models/scaler",
    churn_df=churn_train
)

print(f"_ohe_np shape {_ohe_np.shape}")
print("---"*6)
print(f"_churn_np shape {_churn_np.shape}")

_ohe_np shape (5346, 46)
------------------
_churn_np shape (5346, 1)
